In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.svm import LinearSVC

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym

In [2]:
pd.set_option('display.max_colwidth',-1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [3]:
df = pd.read_csv('data_cleaned.csv',index_col=False)
df.head()

,review_number,facility_ind,security_ind,pricing_ind,location_ind,fb_ind,housekeep_ind,frontoff_ind,Others,cons
0,0,1,0,0,0,0,0,0,0,bedroom bland
1,0,1,0,0,0,0,0,0,0,soft furnish
2,0,1,0,0,0,0,0,0,0,littl furnitur big space
3,0,1,0,0,0,0,0,0,0,uncomfort couch area need colour invent design make want spend time bedroom
4,0,0,0,0,0,1,0,0,0,hotel could resid bar relax drink dinner


In [4]:
# Convert all rows to string
df.cons = df.cons.astype(str)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5788 entries, 0 to 5787
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   review_number  5788 non-null   int64 
 1   facility_ind   5788 non-null   int64 
 2   security_ind   5788 non-null   int64 
 3   pricing_ind    5788 non-null   int64 
 4   location_ind   5788 non-null   int64 
 5   fb_ind         5788 non-null   int64 
 6   housekeep_ind  5788 non-null   int64 
 7   frontoff_ind   5788 non-null   int64 
 8   Others         5788 non-null   int64 
 9   cons           5788 non-null   object
dtypes: int64(9), object(1)
memory usage: 452.3+ KB


In [6]:
cols = ['facility_ind','security_ind','pricing_ind','location_ind','fb_ind','housekeep_ind','frontoff_ind','Others']
print('There are',len(df),'rows in total.')
print(len(df[df[cols].sum(axis=1)>1]),'rows involve more than 1 departments.')
print(len(df[df[cols].sum(axis=1)==1]),'rows involve only 1 departments.')
print(len(df[df[cols].sum(axis=1)==0]),'rows are considered no value.')
print('')
print('Feedback to each department:')
print(df[cols].sum())

There are 5788 rows in total.
423 rows involve more than 1 departments.
5365 rows involve only 1 departments.
0 rows are considered no value.

Feedback to each department:
facility_ind     1988
security_ind     65  
pricing_ind      1273
location_ind     66  
fb_ind           408 
housekeep_ind    180 
frontoff_ind     970 
Others           1296
dtype: int64


In [7]:
Y = df[cols]
X = df['cons']
print(len(Y),len(X))

5788 5788


# SVC

## 1. Facility_ind

In [8]:
y = Y.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=111)
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import FunctionTransformer

max_f = 5000

cv_set = StratifiedKFold(n_splits=3, shuffle=True, random_state=111)

def reshape_for_resampling(X_train):
    return X_train.values.reshape(-1,1)

ros = RandomOverSampler()
rus = RandomUnderSampler()

def ravel_resampled_data(X_train):
    return X_train.ravel()

f_reshape = FunctionTransformer(reshape_for_resampling)
f_ravel = FunctionTransformer(ravel_resampled_data)

cv = CountVectorizer(max_features=max_f, ngram_range=(1,2))

tfidf = TfidfTransformer()

In [10]:
sv = LinearSVC(random_state = 111)

In [11]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.8218143781543928
Mean cross validation f1 score (no resampling): 0.7295999045621785
Test accuracy (no resampling) 0.8376511226252159
Test f1 score (no resampling) 0.7545691906005222

Mean cross validation accuracy (ROS): 0.8127435227564006
Mean cross validation f1 score (ROS): 0.7186163892650734
Test accuracy (ROS) 0.8255613126079447
Test f1 score (ROS) 0.7430025445292622

Mean cross validation accuracy (RUS): 0.7967609864371606
Mean cross validation f1 score (RUS): 0.7285921766454436
Test accuracy (RUS) 0.8143350604490501
Test f1 score (RUS) 0.7443519619500594


### Grid Search

In [12]:
from sklearn.model_selection import GridSearchCV

ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.2s


Best score: 0.821814 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': True}


[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  1.1min finished


### Final prediction

In [13]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=True)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [15]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category facility_ind ...-----------
Test accuracy 0.8376511226252159
              precision    recall  f1-score   support

           0       0.90      0.86      0.88       790
           1       0.73      0.79      0.75       368

    accuracy                           0.84      1158
   macro avg       0.81      0.82      0.82      1158
weighted avg       0.84      0.84      0.84      1158

Test f1: 0.7545691906005222


## 2. Security_Ind

In [16]:
y = Y.iloc[:,1]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [17]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9887687108866494
Mean cross validation f1 score (no resampling): 0.2575757575757575
Test accuracy (no resampling) 0.9905008635578584
Test f1 score (no resampling) 0.35294117647058826

Mean cross validation accuracy (ROS): 0.9831533461607774
Mean cross validation f1 score (ROS): 0.20319865319865316
Test accuracy (ROS) 0.9775474956822107
Test f1 score (ROS) 0.1875

Mean cross validation accuracy (RUS): 0.7308865767402398
Mean cross validation f1 score (RUS): 0.06905446217729874
Test accuracy (RUS) 0.7573402417962003
Test f1 score (RUS) 0.0726072607260726


### Grid Search

In [18]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:   57.2s finished


Best score: 0.990280 using {'countVec__analyzer': 'char_wb', 'countVec__ngram_range': (1, 3), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': True}


### Final Prediction

In [19]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='char_wb',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=True)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [20]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category security_ind ...-----------
Test accuracy 0.9896373056994818
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1156
           1       0.14      1.00      0.25         2

    accuracy                           0.99      1158
   macro avg       0.57      0.99      0.62      1158
weighted avg       1.00      0.99      0.99      1158

Test f1: 0.25


## Pricing_Ind

In [21]:
y = Y.iloc[:,2]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [22]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.929806122026378
Mean cross validation f1 (no resampling): 0.8319572843382367
Test accuracy (no resampling) 0.9300518134715026
Test f1 score (no resampling) 0.8316008316008315

Mean cross validation accuracy (ROS): 0.922245793303537
Mean cross validation f1 (ROS): 0.8271851503235657
Test accuracy (ROS) 0.9214162348877375
Test f1 score (ROS) 0.827977315689981

Mean cross validation accuracy (RUS): 0.9092857094886148
Mean cross validation f1 (RUS): 0.8110738211667826
Test accuracy (RUS) 0.9119170984455959
Test f1 score (RUS) 0.8158844765342962


### Grid Search

In [23]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   26.0s


Best score: 0.935205 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': False}


[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:   52.8s finished


### Final Prediction

In [24]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=False)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [25]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category pricing_ind ...-----------
Test accuracy 0.9317789291882557
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       919
           1       0.82      0.86      0.84       239

    accuracy                           0.93      1158
   macro avg       0.89      0.91      0.90      1158
weighted avg       0.93      0.93      0.93      1158

Test f1: 0.8391038696537678


## Location_Ind

In [26]:
y = Y.iloc[:,3]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [27]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9896326884912306
Mean cross validation f1 (no resampling): 0.2593301435406699
Test accuracy (no resampling) 0.9913644214162349
Test f1 score (no resampling) 0.375

Mean cross validation accuracy (ROS): 0.9889848801820468
Mean cross validation f1 (ROS): 0.37542519063049123
Test accuracy (ROS) 0.9887737478411054
Test f1 score (ROS) 0.380952380952381

Mean cross validation accuracy (RUS): 0.8252649717874584
Mean cross validation f1 (RUS): 0.10181096942604734
Test accuracy (RUS) 0.8160621761658031
Test f1 score (RUS) 0.09361702127659574


### Grid Search

In [28]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
    #print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  1.3min finished


Best score: 0.989633 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 3), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': False}


### Final Prediction

In [29]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=False)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [30]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category location_ind ...-----------
Test accuracy 0.9896373056994818
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1150
           1       0.33      0.50      0.40         8

    accuracy                           0.99      1158
   macro avg       0.66      0.75      0.70      1158
weighted avg       0.99      0.99      0.99      1158

Test f1: 0.4


## FB_Ind

In [31]:
y = Y.iloc[:,4]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [32]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9414691900689167
Mean cross validation f1 (no resampling): 0.4825925925925925
Test accuracy (no resampling) 0.9438687392055267
Test f1 (no resampling) 0.5185185185185185

Mean cross validation accuracy (ROS): 0.9384453384105834
Mean cross validation f1 (ROS): 0.4881040131040131
Test accuracy (ROS) 0.9430051813471503
Test f1 (ROS) 0.5217391304347826

Mean cross validation accuracy (RUS): 0.8652281684402343
Mean cross validation f1 (RUS): 0.4731418998988932
Test accuracy (RUS) 0.8575129533678757
Test f1 (RUS) 0.44816053511705684


### Grid Search

In [33]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  1.6min finished


Best score: 0.938230 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 3), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': False}


### Final Prediction

In [34]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=False)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [35]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category fb_ind ...-----------
Test accuracy 0.9421416234887737
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1101
           1       0.44      0.63      0.52        57

    accuracy                           0.94      1158
   macro avg       0.71      0.79      0.74      1158
weighted avg       0.95      0.94      0.95      1158

Test f1: 0.5179856115107914


## Housekeep_Ind

In [36]:
y = Y.iloc[:,5]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [37]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9768897533795166
Mean cross validation f1 (no resampling): 0.49965112243593257
Test accuracy (no resampling) 0.9810017271157168
Test f1 score (no resampling) 0.5

Mean cross validation accuracy (ROS): 0.9768903130411228
Mean cross validation f1 (ROS): 0.5288838161831195
Test accuracy (ROS) 0.9827288428324698
Test f1 score (ROS) 0.5833333333333333

Mean cross validation accuracy (RUS): 0.7928732970896477
Mean cross validation f1 (RUS): 0.19992979992979995
Test accuracy (RUS) 0.8082901554404145
Test f1 score (RUS) 0.1956521739130435


### Grid Search

In [38]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  1.4min finished


Best score: 0.976890 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 3), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': False}


### Final Prediction

In [39]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=False)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [40]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category housekeep_ind ...-----------
Test accuracy 0.9827288428324698
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1143
           1       0.42      0.93      0.58        15

    accuracy                           0.98      1158
   macro avg       0.71      0.96      0.79      1158
weighted avg       0.99      0.98      0.99      1158

Test f1: 0.5833333333333333


## Front Office_Ind

In [41]:
y = Y.iloc[:,6]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [42]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9198696100389862
Mean cross validation f1 score (no resampling): 0.7401897576779453
Test accuracy (no resampling) 0.9196891191709845
Test f1 score (no resampling) 0.7365439093484418

Mean cross validation accuracy (ROS): 0.914685464580696
Mean cross validation f1 score (ROS): 0.7390735847330916
Test accuracy (ROS) 0.9101899827288429
Test f1 score (ROS) 0.7291666666666665

Mean cross validation accuracy (RUS): 0.8790484521438957
Mean cross validation f1 score (RUS): 0.7019274649499708
Test accuracy (RUS) 0.8713298791018999
Test f1 score (RUS) 0.6795698924731183


### Grid Search

In [43]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.5s


Best score: 0.920734 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'svc__C': 1, 'svc__penalty': 'l2', 'tfidf__use_idf': False}


[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  1.0min finished


### Final Prediction

In [44]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=False)
sv = LinearSVC(random_state = 111,C=1,penalty='l2')
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [45]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category frontoff_ind ...-----------
Test accuracy 0.9205526770293609
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       996
           1       0.68      0.81      0.74       162

    accuracy                           0.92      1158
   macro avg       0.83      0.87      0.85      1158
weighted avg       0.93      0.92      0.92      1158

Test f1: 0.7401129943502827


## Others

In [46]:
y = Y.iloc[:,7]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [47]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.8254863179527131
Mean cross validation f1 score (no resampling): 0.5540955869527298
Test accuracy (no resampling) 0.8281519861830743
Test f1 score (no resampling) 0.5567928730512249

Mean cross validation accuracy (ROS): 0.8192211861020353
Mean cross validation f1 score (ROS): 0.5844112739588755
Test accuracy (ROS) 0.803972366148532
Test f1 score (ROS) 0.5609284332688588

Mean cross validation accuracy (RUS): 0.7488097396790004
Mean cross validation f1 score (RUS): 0.5723042847077496
Test accuracy (RUS) 0.7633851468048359
Test f1 score (RUS) 0.5970588235294119


### Grid Search

In [48]:
ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'svc__C': [0.001,0.01,0.1,1,10,20],
    'svc__penalty':['l1','l2']
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:   35.4s


Best score: 0.768678 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'svc__C': 0.001, 'svc__penalty': 'l2', 'tfidf__use_idf': True}


[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:   39.0s finished


### Final Prediction

In [49]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=True)
sv = LinearSVC(random_state = 111,C=0.001,penalty='l2')
ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('svc',sv)])

In [50]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category Others ...-----------
Test accuracy 0.7746113989637305
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       811
           1       0.67      0.50      0.57       347

    accuracy                           0.77      1158
   macro avg       0.74      0.69      0.71      1158
weighted avg       0.76      0.77      0.76      1158

Test f1: 0.5685950413223141


# KNN

## Facility_Ind

In [51]:
y = Y.iloc[:,0]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [52]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [53]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.7012958684660907
Mean cross validation f1 score (no resampling): 0.3022724335593301
Test accuracy (no resampling) 0.7012089810017271
Test f1 score (no resampling) 0.31620553359683795

Mean cross validation accuracy (ROS): 0.7049675284336079
Mean cross validation f1 score (ROS): 0.32573599240265905
Test accuracy (ROS) 0.7236614853195165
Test f1 score (ROS) 0.4444444444444444

Mean cross validation accuracy (RUS): 0.696760650640197
Mean cross validation f1 score (RUS): 0.32558637582390976
Test accuracy (RUS) 0.690846286701209
Test f1 score (RUS) 0.39730639730639733


### Grid Search

In [55]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.6s
/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  2.9min finished


Best score: 0.762637 using {'countVec__analyzer': 'char_wb', 'countVec__ngram_range': (1, 3), 'knn__n_neighbors': 4, 'tfidf__use_idf': True}


### Final Prediction

In [56]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='char_wb',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=True)
knn = KNeighborsClassifier(n_neighbors=4)
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [57]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category facility_ind ...-----------
Test accuracy 0.7659758203799655
              precision    recall  f1-score   support

           0       0.83      0.82      0.82       771
           1       0.65      0.66      0.65       387

    accuracy                           0.77      1158
   macro avg       0.74      0.74      0.74      1158
weighted avg       0.77      0.77      0.77      1158

Test f1: 0.6547770700636942


## Security_Ind

In [58]:
y = Y.iloc[:,1]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [59]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9885526815066538
Mean cross validation f1 score (no resampling): 0.06666666666666667
Test accuracy (no resampling) 0.9887737478411054
Test f1 score (no resampling) 0.13333333333333333

Mean cross validation accuracy (ROS): 0.9440619623750696
Mean cross validation f1 score (ROS): 0.1348515779316628
Test accuracy (ROS) 0.9550949913644214
Test f1 score (ROS) 0.21212121212121213

Mean cross validation accuracy (RUS): 0.4525275437459495
Mean cross validation f1 score (RUS): 0.035087716073057246
Test accuracy (RUS) 0.4231433506044905
Test f1 score (RUS) 0.03468208092485549


### Grid Search

In [60]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  3.5min finished


Best score: 0.988985 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'knn__n_neighbors': 2, 'tfidf__use_idf': False}


### Final Prediction

In [61]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=False)
knn = KNeighborsClassifier(n_neighbors=2)
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [62]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category security_ind ...-----------
Test accuracy 0.9887737478411054
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1157
           1       0.07      1.00      0.13         1

    accuracy                           0.99      1158
   macro avg       0.54      0.99      0.56      1158
weighted avg       1.00      0.99      0.99      1158

Test f1: 0.13333333333333333


## Pricing_Ind

In [63]:
y = Y.iloc[:,2]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [64]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 score (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 score (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 score (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 score (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.8829362814068102
Mean cross validation f1 score (no resampling): 0.6468351697363345
Test accuracy (no resampling) 0.8816925734024179
Test f1 score (no resampling) 0.6366047745358089

Mean cross validation accuracy (ROS): 0.8786163933839043
Mean cross validation f1 score (ROS): 0.6432709159779053
Test accuracy (ROS) 0.8721934369602763
Test f1 score (ROS) 0.6243654822335025

Mean cross validation accuracy (RUS): 0.8634940569534036
Mean cross validation f1 score (RUS): 0.6468089689906749
Test accuracy (RUS) 0.8687392055267703
Test f1 score (RUS) 0.6


### Grid Search

In [65]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  1.7min finished


Best score: 0.901728 using {'countVec__analyzer': 'char_wb', 'countVec__ngram_range': (1, 3), 'knn__n_neighbors': 6, 'tfidf__use_idf': True}


### Final Prediction

In [66]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='char_wb',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=True)
knn = KNeighborsClassifier(n_neighbors=6)
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [67]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category pricing_ind ...-----------
Test accuracy 0.9058721934369602
              precision    recall  f1-score   support

           0       0.97      0.92      0.94       955
           1       0.69      0.85      0.76       203

    accuracy                           0.91      1158
   macro avg       0.83      0.88      0.85      1158
weighted avg       0.92      0.91      0.91      1158

Test f1: 0.7604395604395604


# Location_Ind

In [68]:
y = Y.iloc[:,3]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [69]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9881209025774655
Mean cross validation f1 (no resampling): 0.0
Test accuracy (no resampling) 0.9913644214162349
Test f1 (no resampling) 0.2857142857142857

Mean cross validation accuracy (ROS): 0.969330543979888
Mean cross validation f1 (ROS): 0.17457450011511963
Test accuracy (ROS) 0.9758203799654577
Test f1 (ROS) 0.33333333333333337

Mean cross validation accuracy (RUS): 0.5350178867849343
Mean cross validation f1 (RUS): 0.04604639210835221
Test accuracy (RUS) 0.4835924006908463
Test f1 (RUS) 0.02922077922077922


### Grid Search

In [70]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.2s
/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  3.5min finished


Best score: 0.988985 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 3), 'knn__n_neighbors': 2, 'tfidf__use_idf': True}


### Final Prediction

In [71]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=True)
knn = KNeighborsClassifier(n_neighbors=2)
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [72]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category location_ind ...-----------
Test accuracy 0.9905008635578584
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1157
           1       0.08      1.00      0.15         1

    accuracy                           0.99      1158
   macro avg       0.54      1.00      0.57      1158
weighted avg       1.00      0.99      0.99      1158

Test f1: 0.15384615384615385


# FB_Ind

In [73]:
y = Y.iloc[:,4]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [74]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.9291578939709894
Mean cross validation f1 (no resampling): 0.062403368904916885
Test accuracy (no resampling) 0.9300518134715026
Test f1 (no resampling) 0.06896551724137931

Mean cross validation accuracy (ROS): 0.9302374812093616
Mean cross validation f1 (ROS): 0.09506872608476886
Test accuracy (ROS) 0.9317789291882557
Test f1 (ROS) 0.09195402298850576

Mean cross validation accuracy (RUS): 0.8736552730756876
Mean cross validation f1 (RUS): 0.2125055531439045
Test accuracy (RUS) 0.9291882556131261
Test f1 (RUS) 0.046511627906976744


### Grid Search

In [75]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.8s
/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  3.4min finished


Best score: 0.931966 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'knn__n_neighbors': 2, 'tfidf__use_idf': False}


### Final Prediction

In [76]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=False)
knn = KNeighborsClassifier(n_neighbors=2)
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [77]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category fb_ind ...-----------
Test accuracy 0.9343696027633851
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      1146
           1       0.11      0.75      0.19        12

    accuracy                           0.93      1158
   macro avg       0.55      0.84      0.58      1158
weighted avg       0.99      0.93      0.96      1158

Test f1: 0.19148936170212766


# Housekeep_Ind

In [78]:
y = Y.iloc[:,5]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [79]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.8528911279084214
Mean cross validation f1 (no resampling): 0.12297965367738417
Test accuracy (no resampling) 0.9697754749568221
Test f1 (no resampling) 0.186046511627907

Mean cross validation accuracy (ROS): 0.9669547804615416
Mean cross validation f1 (ROS): 0.17228195937873358
Test accuracy (ROS) 0.9715025906735751
Test f1 (ROS) 0.15384615384615385

Mean cross validation accuracy (RUS): 0.8788371798875527
Mean cross validation f1 (RUS): 0.2288668413997743
Test accuracy (RUS) 0.8791018998272885
Test f1 (RUS) 0.22222222222222224


### Grid Search

In [80]:
ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.0s


Best score: 0.892875 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 3), 'knn__n_neighbors': 2, 'tfidf__use_idf': False}


[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:   40.2s finished


### Final Prediction

In [81]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,3))
tfidf = TfidfTransformer(use_idf=False)
knn = KNeighborsClassifier(n_neighbors=2)
ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [82]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category housekeep_ind ...-----------
Test accuracy 0.917098445595855
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1053
           1       0.64      0.20      0.30       105

    accuracy                           0.92      1158
   macro avg       0.78      0.59      0.63      1158
weighted avg       0.90      0.92      0.90      1158

Test f1: 0.30434782608695654


# Front Office_Ind

In [83]:
y = Y.iloc[:,6]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [84]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.8708426936736972
Mean cross validation f1 (no resampling): 0.40306811930713166
Test accuracy (no resampling) 0.8670120898100173
Test f1 (no resampling) 0.36885245901639346

Mean cross validation accuracy (ROS): 0.8714899423212749
Mean cross validation f1 (ROS): 0.45740167918692004
Test accuracy (ROS) 0.8713298791018999
Test f1 (ROS) 0.4808362369337979

Mean cross validation accuracy (RUS): 0.8488088162373503
Mean cross validation f1 (RUS): 0.4586941922005802
Test accuracy (RUS) 0.8747841105354058
Test f1 (RUS) 0.4569288389513108


### Grid Search

In [85]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  3.1min finished


Best score: 0.873866 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'knn__n_neighbors': 3, 'tfidf__use_idf': False}


### Final Prediction

In [86]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=False)
knn = KNeighborsClassifier(n_neighbors=3)
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [87]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category frontoff_ind ...-----------
Test accuracy 0.8704663212435233
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      1054
           1       0.38      0.70      0.49       104

    accuracy                           0.87      1158
   macro avg       0.67      0.79      0.71      1158
weighted avg       0.92      0.87      0.89      1158

Test f1: 0.49324324324324326


# Others

In [88]:
y = Y.iloc[:,7]
train_index = X_train.index
test_index = X_test.index
y_train = Y.loc[train_index,y.name]
y_test = Y.loc[test_index,y.name]
print(len(X_train), len(X_test), len(y_train), len(y_test))

4630 1158 4630 1158


In [89]:
ppl = Pipeline([('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (no resampling):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (no resampling):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (no resampling)',accuracy_score(y_pred, y_test))
print('Test f1 (no resampling)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (ROS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (ROS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (ROS)',accuracy_score(y_pred, y_test))
print('Test f1 (ROS)',f1_score(y_pred, y_test))
print('')

ppl = Pipeline([('reshape',f_reshape),('resample',rus),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])
val_score = cross_validate(ppl, X_train, y_train, cv=cv_set, scoring=['accuracy','f1'])
print('Mean cross validation accuracy (RUS):', val_score['test_accuracy'].mean())
print('Mean cross validation f1 (RUS):', val_score['test_f1'].mean())
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy (RUS)',accuracy_score(y_pred, y_test))
print('Test f1 (RUS)',f1_score(y_pred, y_test))

Mean cross validation accuracy (no resampling): 0.6315407651917345
Mean cross validation f1 (no resampling): 0.42033297707645517
Test accuracy (no resampling) 0.8074265975820379
Test f1 (no resampling) 0.39237057220708443

Mean cross validation accuracy (ROS): 0.7950330312279984
Mean cross validation f1 (ROS): 0.4338840860200044
Test accuracy (ROS) 0.8031088082901554
Test f1 (ROS) 0.4597156398104265

Mean cross validation accuracy (RUS): 0.4559512736219172
Mean cross validation f1 (RUS): 0.43022491322370104
Test accuracy (RUS) 0.44127806563039723
Test f1 (RUS) 0.4299559471365638


### Grid Search

In [90]:
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

param_grid = {
    'countVec__ngram_range': [(1,2), (1,3)],
    'countVec__analyzer': ['word','char_wb'],
    'tfidf__use_idf': [True, False],
    'knn__n_neighbors': [2,3,4,5,6,7,8],
}

kfold_splits = 5
grid = GridSearchCV(estimator=ppl,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=cv_set,
                    param_grid=param_grid,)
#grid.get_params

grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.3s


Best score: 0.800004 using {'countVec__analyzer': 'word', 'countVec__ngram_range': (1, 2), 'knn__n_neighbors': 2, 'tfidf__use_idf': False}


[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  2.4min finished


### Final Prediction

In [91]:
# Definition of the best model
cv = CountVectorizer(max_features=max_f, analyzer='word',ngram_range=(1,2))
tfidf = TfidfTransformer(use_idf=False)
knn = KNeighborsClassifier(n_neighbors=2)
ppl = Pipeline([('reshape',f_reshape),('resample',ros),('ravel',f_ravel),('countVec',cv),('tfidf',tfidf),('knn',knn)])

In [92]:
print('-------------Predicting category',y.name,'...-----------')
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)
print('Test accuracy',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print('Test f1:',f1_score(y_pred, y_test))

-------------Predicting category Others ...-----------
Test accuracy 0.809153713298791
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      1043
           1       0.29      0.66      0.41       115

    accuracy                           0.81      1158
   macro avg       0.63      0.74      0.65      1158
weighted avg       0.89      0.81      0.84      1158

Test f1: 0.4075067024128686
